# Projeto 2 - Ciência dos Dados

Nome: _____

Nome: Diogo Nobre de Araujo Cintra

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [6]:
%%capture
!pip install tweepy

In [7]:
%matplotlib inline
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import matplotlib.pyplot as plt
from numpy import arange
from numpy import percentile
import numpy as np
import os
import re 

___
## Autenticando no  Twitter

* Conta: ***@Ciencia dos dados***

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


Capturando os dados do twitter:

Salvando os dados em uma planilha Excel:

___
### Classificando as mensagens na coragem

#### 0 = irrelevante
#### 1 = relevante

In [8]:
TREINAMENTO = pd.read_excel('Correios.xlsx',sheet_name = 'Treinamento')
TESTE = pd.read_excel('Correios.xlsx',sheet_name = 'Teste')

In [9]:
TREINAMENTO_RELEVANTE = TREINAMENTO.loc[TREINAMENTO["Classificacao"]==1]
TREINAMENTO_IRRELEVANTE = TREINAMENTO.loc[TREINAMENTO["Classificacao"]==0]

def clean(text):
    punctuation = '[!\-.:?;#$%&*_1234567890"]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [10]:
texto_relevante = " ".join(TREINAMENTO_RELEVANTE['Treinamento'])
texto_relevante_1 = clean(texto_relevante)
texto_relevante_2 = texto_relevante_1.split()
texto_relevante_3 = []

texto_irrelevante = " ".join(TREINAMENTO_IRRELEVANTE['Treinamento'])
texto_irrelevante_1 = clean(texto_irrelevante)
texto_irrelevante_2 = texto_irrelevante_1.split()
texto_irrelevante_3 = []

treinamento = " ".join(TREINAMENTO['Treinamento'])
treinamento_1 = clean(treinamento)
treinamento_2 = treinamento_1.split()
treinamento_3 = []

for e in range(len(texto_relevante_2)-1):
    if texto_relevante_2[e] != 'rt' and texto_relevante_2[e][0] != '@':     
        texto_relevante_3.append(texto_relevante_2[e])    
        
for e in range(len(texto_irrelevante_2)-1):
    if texto_irrelevante_2[e] != 'rt' and texto_irrelevante_2[e][0] != '@':     
        texto_irrelevante_3.append(texto_irrelevante_2[e]) 
        
for e in range(len(treinamento_2)-1):
    if treinamento_2[e] != 'rt' and treinamento_2[e][0] != '@':     
        treinamento_3.append(treinamento_2[e])
        
texto_relevante_4 = pd.Series(texto_relevante_3)
texto_irrelevante_4 = pd.Series(texto_irrelevante_3)
treinamento_4 = pd.Series(treinamento_3)

#### Probabilidade do tweet ser relevante

In [28]:
prob_relevante = len(TREINAMENTO.loc[TREINAMENTO["Classificacao"]==1])/len(TREINAMENTO["Treinamento"])
prob_relevante

0.4013377926421405

#### Probabilidade do tweet ser relevante

In [29]:
prob_irrelevante = len(TREINAMENTO.loc[TREINAMENTO["Classificacao"]==0])/len(TREINAMENTO["Treinamento"])
prob_irrelevante

0.5986622073578596

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

#contador das palavras
def conta_palavra(texto, dic):
    texto_sem_pontuacao = clean(texto)
    palavras_do_texto = texto_sem_pontuacao.split()
    palavras=dic
    for p in palavras_do_texto:
        if p in palavras:
            palavras[p]+=1
        else:
            palavras[p]=1
    return palavras

In [13]:
#associa um valor de probabilidade
#probrev={}
#formato {palavra:valor}
#probnorev={}
#formato {palavra:valor}

total_amostral = len(treinamento_4)
total_relevante = texto_relevante_4.value_counts()
total_irrelevante = texto_irrelevante_4.value_counts()

#Probabilidade da palavra ser relevante
prob_relevante_palavra = total_relevante/total_amostral
prob_relevante_palavra

#Probabilidade da palavra ser irrelevante
prob_irrelevante_palavra = total_irrelevante/total_amostral
prob_irrelevante_palavra

correios         0.021303
de               0.018936
o                0.016412
que              0.016254
a                0.013255
os               0.010415
dos              0.009153
e                0.007732
//t              0.007101
https            0.007101
uma              0.006943
em               0.006786
do               0.006154
é                0.005997
no               0.005839
na               0.005365
não              0.005208
correios,        0.004103
da               0.004103
se               0.003945
governo          0.003629
transformar      0.003472
ao               0.003472
sobre            0.003472
eu               0.003314
privatização     0.003156
para             0.003156
com              0.003156
amazon           0.002998
por              0.002840
                   ...   
delação          0.000158
esquerd…         0.000158
cujo             0.000158
volta            0.000158
servidores       0.000158
td               0.000158
não,             0.000158
álbuns      

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [1]:
smoth = 10**(-10) #é uma estimativa e deve mudar

# defini probabilidade de ser relevante:
def relevante(tweet):
    t = clean(tweet)
    t = t.split()
    txt = []
    for e in range(len(t)-1):
        if t[e] != 'rt' and t[e][0] != '@':     
            txt.append(t[e])
    prob = 1
    for p in txt:
        if p in prob_relevante_palavra:
            prob = prob*(prob_relevante_palavra[p]+smoth)
        else:
            prob = prob*(smoth)
    prob = prob * prob_relevante
    return prob

# defini probabilidade de ser irrelevante:
def irrelevante(tweet):
    t = clean(tweet)
    t = t.split()
    txt = []
    for e in range(len(t)-1):
        if t[e] != 'rt' and t[e][0] != '@':     
            txt.append(t[e])
    prob = 1
    for p in txt:
        if p in prob_irrelevante_palavra:
            prob = prob*(prob_irrelevante_palavra[p]+smoth)
        else:
            prob = prob*(smoth)
    prob = prob * prob_irrelevante
    return prob

In [44]:
def compara (tweet):
    r = relevante(tweet)
    nor = irrelevante(tweet)
    return r>nor

In [45]:
prob_relevante_teste = len(TESTE.loc[TESTE["Classificacao"]==1])/len(TESTE["Teste"])
prob_irrelevante_teste = len(TESTE.loc[TESTE["Classificacao"]==0])/len(TESTE["Teste"])

In [46]:
rev = 0
norev = 0
for txt in TESTE["Teste"]:
    if compara (txt):
        rev +=1
    else:
        norev+=1
print ("a probabilidade de ser relevante é {}".format(rev/TESTE["Teste"].size))
print ("a probabilidade de não ser relevante é {}".format(norev/TESTE["Teste"].size))
if rev/(TESTE["Teste"].size) < 0.9*prob_relevante_teste:
    print ("poucos relevantes {} vezes o esperado".format (rev/TESTE["Teste"].size/prob_relevante_teste))
elif rev/(TESTE["Teste"].size) > 1.1*prob_relevante_teste:
    print ("muitos relevantes {} vezes o esperado" .format (rev/TESTE["Teste"].size/prob_relevante_teste))
else:
    print ("ok: {} veze o esperado".format (rev/TESTE["Teste"].size/prob_relevante))


a probabilidade de ser relevante é 0.24
a probabilidade de não ser relevante é 0.76
poucos relevantes 0.5393258426966292 vezes o esperado


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**